<a href="https://colab.research.google.com/github/jmarrietar/ocular/blob/master/notebooks/PAWS_DEBUG_200000_(Work_in_Progress_incorporar_PAWS_de_a_poco)_Copy_of_PyTorch_XLA_ResNet18_CIFAR10_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -U PyYAML

     |████████████████████████████████| 636 kB 5.3 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
pwd

'/content'

In [3]:
!git clone -b feature/DR-images-v2 https://github.com/jmarrietar/suncet.git

Cloning into 'suncet'...
remote: Enumerating objects: 336, done.
remote: Counting objects: 100% (336/336), done.
remote: Compressing objects: 100% (214/214), done.
remote: Total 336 (delta 199), reused 250 (delta 119), pack-reused 0
Receiving objects: 100% (336/336), 1.11 MiB | 9.06 MiB/s, done.
Resolving deltas: 100% (199/199), done.


In [7]:
cd suncet

/content/suncet


In [5]:
!mkdir datasets
!mkdir datasets/dr
!mkdir logs

In [6]:
!python download.py -d sample@2000

Downloading...
From: https://drive.google.com/uc?id=1PB7uGd-dUnZKnKZpZl-HvE1DVcWgX50F
To: /content/suncet/datasets/dr/sample@2000.zip
214MB [00:02, 96.1MB/s]


In [5]:
import argparse
import yaml

In [6]:
parser = argparse.ArgumentParser()
parser.add_argument(
    '--fname', type=str,
    help='name of config file to load',
    default='configs.yaml')
parser.add_argument(
    '--devices', type=str, nargs='+', default=['cuda:0'],
    help='which devices to use on local machine')
parser.add_argument(
    '--sel', type=str,
    help='which script to run',
    choices=[
        'paws_train',
        'suncet_train',
        'fine_tune',
        'snn_fine_tune'
    ])

_StoreAction(option_strings=['--sel'], dest='sel', nargs=None, const=None, default=None, type=<class 'str'>, choices=['paws_train', 'suncet_train', 'fine_tune', 'snn_fine_tune'], help='which script to run', metavar=None)

In [8]:
args = parser.parse_args(['--sel', 'paws_train',
                            '--fname', 'configs/paws/dr_train.yaml'
])

In [9]:
fname = args.fname
sel = args.sel

In [10]:
import pprint
import logging
logging.basicConfig()
logger = logging.getLogger()

logger.info(f'called-params {sel} {fname}')

In [11]:
# -- load script params
params = None
with open(fname, 'r') as y_file:
    #params = yaml.load(y_file, Loader=yaml.FullLoader)
    params = yaml.load(y_file)
    logger.info('loaded params...')
    #if rank == 0:
    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(params)

{   'criterion': {   'classes_per_batch': 2,
                     'me_max': True,
                     'sharpen': 0.25,
                     'supervised_imgs_per_class': 32,
                     'supervised_views': 1,
                     'temperature': 0.1,
                     'unsupervised_batch_size': 32},
    'data': {   'color_jitter_strength': 1.0,
                'data_seed': None,
                'dataset': 'dr',
                'label_smoothing': 0.1,
                'multicrop': 6,
                'normalize': True,
                'root_path': 'datasets/',
                's_image_folder': 'dr/sample@2000/',
                'subset_path': 'dr_subsets',
                'u_image_folder': 'dr/sample@2000/',
                'unique_classes_per_rank': False,
                'unlabeled_frac': 0.9},
    'logging': {'folder': 'logs/', 'write_tag': 'paws'},
    'meta': {   'copy_data': True,
                'device': 'cuda:0',
                'load_checkpoint': False,
              

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """


In [12]:
args = params

In [13]:
import os

In [14]:
# Define Parameters
FLAGS = {}
FLAGS['data_dir'] = "/tmp/cifar"
FLAGS['batch_size'] = 32
FLAGS['num_workers'] = 2
FLAGS['learning_rate'] = 0.02
FLAGS['momentum'] = 0.9
FLAGS['num_epochs'] = 10
FLAGS['num_cores'] = 8 if os.environ.get('TPU_NAME', None) else 1
FLAGS['log_steps'] = 20
FLAGS['metrics_debug'] = False

In [15]:
FLAGS['model_name'] = args["meta"]["model_name"]

In [16]:
model_name = args["meta"]["model_name"]

In [17]:
output_dim = args["meta"]["output_dim"]
multicrop = args["data"]["multicrop"]

In [18]:
# -- CRITERTION
reg = args["criterion"]["me_max"]
supervised_views = args["criterion"]["supervised_views"]
classes_per_batch = args["criterion"]["classes_per_batch"]
s_batch_size = args["criterion"]["supervised_imgs_per_class"]
u_batch_size = args["criterion"]["unsupervised_batch_size"]
temperature = args["criterion"]["temperature"]
sharpen = args["criterion"]["sharpen"]

In [19]:
# -- DATA
unlabeled_frac = args["data"]["unlabeled_frac"]
color_jitter = args["data"]["color_jitter_strength"]
normalize = args["data"]["normalize"]
root_path = args["data"]["root_path"]
s_image_folder = args["data"]["s_image_folder"]
u_image_folder = args["data"]["u_image_folder"]
dataset_name = args["data"]["dataset"]
subset_path = args["data"]["subset_path"]
unique_classes = args["data"]["unique_classes_per_rank"]
label_smoothing = args["data"]["label_smoothing"]
data_seed = None

In [20]:
crop_scale = (0.14, 1.0) if multicrop > 0 else (0.08, 1.0)
mc_scale = (0.05, 0.14)
mc_size = 96

In [21]:
copy_data = args["meta"]["copy_data"]
use_pred_head = args["meta"]["use_pred_head"]

In [22]:
use_fp16 = args["meta"]["use_fp16"]

In [23]:
# -- OPTIMIZATION
wd = float(args["optimization"]["weight_decay"])
num_epochs = args["optimization"]["epochs"]
warmup = args["optimization"]["warmup"]
start_lr = args["optimization"]["start_lr"]
lr = args["optimization"]["lr"]
final_lr = args["optimization"]["final_lr"]
mom = args["optimization"]["momentum"]
nesterov = args["optimization"]["nesterov"]

In [ ]:
# ----------------------------------------------------------------------- #
#  PASSED IN PARAMS FROM CONFIG FILE
# ----------------------------------------------------------------------- #
# -- META


load_model = args["meta"]["load_checkpoint"]
r_file = args["meta"]["read_checkpoint"]


#device = torch.device(args["meta"]["device"])
device = xm.xla_device()




# -- LOGGING
folder = args["logging"]["folder"]
tag = args["logging"]["write_tag"]
# ----------------------------------------------------------------------- #

# -- init torch distributed backend
#world_size, rank = init_distributed()


In [24]:
import logging
import sys
from collections import OrderedDict

import numpy as np

import torch

import src.resnet as resnet
import src.wide_resnet as wide_resnet
from src.utils import (
    gpu_timer,
    init_distributed,
    WarmupCosineSchedule,
    CSVLogger,
    AverageMeter,
)
from src.losses import init_paws_loss, make_labels_matrix
from src.data_manager import init_data, make_transforms, make_multicrop_transform
from src.sgd import SGD
from src.lars import LARS

import torchvision.models as models
from torch.hub import load_state_dict_from_url

#import apex
from torch.nn.parallel import DistributedDataParallel

In [25]:
def load_checkpoint(r_path, encoder, opt, scaler, use_fp16=False):
    checkpoint = torch.load(r_path, map_location="cpu")
    epoch = checkpoint["epoch"]

    # -- loading encoder
    encoder.load_state_dict(checkpoint["encoder"])
    logger.info(f"loaded encoder from epoch {epoch}")

    # -- loading optimizer
    opt.load_state_dict(checkpoint["opt"])
    if use_fp16:
        scaler.load_state_dict(checkpoint["amp"])
    logger.info(f"loaded optimizers from epoch {epoch}")
    logger.info(f"read-path: {r_path}")
    del checkpoint
    return encoder, opt, epoch


def init_model(device, model_name="resnet50", use_pred=False, output_dim=128):
    if "wide_resnet" in model_name:
        encoder = wide_resnet.__dict__[model_name](dropout_rate=0.0)
        hidden_dim = 128
    else:
        encoder = resnet.__dict__[model_name]() 

        # Load pre-trained ResNetImagenNet
        #logger.info("Load pre-trained ResNet ImagenNet weigths ...")
        #state_dict = load_state_dict_from_url('https://download.pytorch.org/models/resnet50-0676ba61.pth',progress=True)
        #log = encoder.load_state_dict(state_dict, strict=False)
        #logger.info(log)
    
        hidden_dim = 2048
        if "w2" in model_name:
            hidden_dim *= 2
        elif "w4" in model_name:
            hidden_dim *= 4

    # -- projection head
    encoder.fc = torch.nn.Sequential(
        OrderedDict(
            [
                ("fc1", torch.nn.Linear(hidden_dim, hidden_dim)),
                ("bn1", torch.nn.BatchNorm1d(hidden_dim)),
                ("relu1", torch.nn.ReLU(inplace=True)),
                ("fc2", torch.nn.Linear(hidden_dim, hidden_dim)),
                ("bn2", torch.nn.BatchNorm1d(hidden_dim)),
                ("relu2", torch.nn.ReLU(inplace=True)),
                ("fc3", torch.nn.Linear(hidden_dim, output_dim)),
            ]
        )
    )

    # -- prediction head
    encoder.pred = None
    if use_pred:
        mx = 4  # 4x bottleneck prediction head
        pred_head = OrderedDict([])
        pred_head["bn1"] = torch.nn.BatchNorm1d(output_dim)
        pred_head["fc1"] = torch.nn.Linear(output_dim, output_dim // mx)
        pred_head["bn2"] = torch.nn.BatchNorm1d(output_dim // mx)
        pred_head["relu"] = torch.nn.ReLU(inplace=True)
        pred_head["fc2"] = torch.nn.Linear(output_dim // mx, output_dim)
        encoder.pred = torch.nn.Sequential(pred_head)

    encoder.to(device)
    logger.info(encoder)
    return encoder


def init_opt(
    encoder,
    iterations_per_epoch,
    start_lr,
    ref_lr,
    ref_mom,
    nesterov,
    warmup,
    num_epochs,
    weight_decay=1e-6,
    final_lr=0.0,
):
    param_groups = [
        {
            "params": (
                p
                for n, p in encoder.named_parameters()
                if ("bias" not in n) and ("bn" not in n)
            )
        },
        {
            "params": (
                p for n, p in encoder.named_parameters() if ("bias" in n) or ("bn" in n)
            ),
            "LARS_exclude": True,
            "weight_decay": 0,
        },
    ]
    optimizer = SGD(
        param_groups,
        weight_decay=weight_decay,
        momentum=0.9,
        nesterov=nesterov,
        lr=ref_lr,
    )
    scheduler = WarmupCosineSchedule(
        optimizer,
        warmup_steps=warmup * iterations_per_epoch,
        start_lr=start_lr,
        ref_lr=ref_lr,
        final_lr=final_lr,
        T_max=num_epochs * iterations_per_epoch,
    )
    optimizer = LARS(optimizer, trust_coefficient=0.001)
    return encoder, optimizer, scheduler

## PyTorch/XLA ResNet18/CIFAR10 (GPU or TPU)

### [RUNME] Install Colab compatible PyTorch/XLA wheels and dependencies



In [1]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8.1-cp37-cp37m-linux_x86_64.whl

  Using cached https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8.1-cp37-cp37m-linux_x86_64.whl (145.0 MB)


In [2]:
!pip uninstall torch -y
!pip install torch==1.8.1

Found existing installation: torch 1.8.1
Uninstalling torch-1.8.1:
  Successfully uninstalled torch-1.8.1
  Using cached torch-1.8.1-cp37-cp37m-manylinux1_x86_64.whl (804.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torch 1.8.1 which is incompatible.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.8.1 which is incompatible.


In [3]:
import argparse
import logging
import os
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.debug.metrics as met
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.utils.serialization as xser
import torch_xla.utils.utils as xu
import torchvision
from torchvision import datasets, transforms

In [4]:
import gdown

In [ ]:
def download(data, url):
    # Download dataset
    import zipfile
    url = url
    output = "{}.zip".format(data)
    gdown.download(url, output, quiet=False)

    # Uncompress dataset
    local_zip = '{}.zip'.format(data)
    zip_ref = zipfile.ZipFile(local_zip, "r")
    zip_ref.extractall()
    zip_ref.close()

In [26]:
data_samples = {
    "sample@200": "https://drive.google.com/uc?id=1FfV7YyDJvNUCDP5r3-8iQfZ2-xJp_pgb",
    "sample@500": "https://drive.google.com/uc?id=1dHwUqpmSogEdjAB9rwDUL-OKFRUcVXte",
    "sample@1000": "https://drive.google.com/uc?id=1DPZrHrj3Bdte5Dc6NCZ33CAqMG-Oipa2",
    "sample@2000": "https://drive.google.com/uc?id=1PB7uGd-dUnZKnKZpZl-HvE1DVcWgX50F",
    "sample@3000": "https://drive.google.com/uc?id=1_yre5K9YYvJgSrT4xvrI8eD_htucIywA",
    "sample@4000_images": "https://drive.google.com/uc?id=1dqVB8EozEpwWzyuU80AauoQmsiw3Gtm2",
    "sample@20000": "https://drive.google.com/uc?id=1MTDpLzpmhSiZq2jSdmHx2UDPn9FC8gzO",
    "val-voets-tf": "https://drive.google.com/uc?id=1VzVgMGTkBBPG2qbzLunD9HvLzH6tcyrv",
    "train_voets": "https://drive.google.com/uc?id=1AmcFh1MOOZ6aqKm2eO7XEdgmIEqHKTZ5",
    "voets_test_images": "https://drive.google.com/uc?id=15S_V3B_Z3BOjCT3AbO2c887FyS5B0Lyd"
}

In [27]:
UNLABELED = 'sample@2000'

In [ ]:
URL_UNLABELED = data_samples[UNLABELED]
download(UNLABELED, URL_UNLABELED)

Only run the below commented cell if you would like a nightly release

In [ ]:
# VERSION = "nightly"  #@param ["nightly", "20200516"]  # or YYYYMMDD format
# !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --version $VERSION
# import os 
# os.environ['LD_LIBRARY_PATH']='/usr/local/lib'
# !echo $LD_LIBRARY_PATH

# !sudo ln -s /usr/local/lib/libmkl_intel_lp64.so /usr/local/lib/libmkl_intel_lp64.so.1
# !sudo ln -s /usr/local/lib/libmkl_intel_thread.so /usr/local/lib/libmkl_intel_thread.so.1
# !sudo ln -s /usr/local/lib/libmkl_core.so /usr/local/lib/libmkl_core.so.1

# !ldconfig
# !ldd /usr/local/lib/python3.7/dist-packages/torch/lib/libtorch.so

In [28]:
# PyTorch/XLA GPU Setup (only if GPU runtime)
import os
if os.environ.get('COLAB_GPU', '0') == '1':
  os.environ['GPU_NUM_DEVICES'] = '1'
  os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/usr/local/cuda/'

### Define Parameters



In [29]:
# Result Visualization Helper
from matplotlib import pyplot as plt

M, N = 4, 6
RESULT_IMG_PATH = '/tmp/test_result.jpg'
CIFAR10_LABELS = ['airplane', 'automobile', 'bird', 'cat', 'deer',
                 'dog', 'frog', 'horse', 'ship', 'truck']

def plot_results(images, labels, preds):
  images, labels, preds = images[:M*N], labels[:M*N], preds[:M*N]
  inv_norm = transforms.Normalize(
      mean=(-0.4914/0.2023, -0.4822/0.1994, -0.4465/0.2010),
      std=(1/0.2023, 1/0.1994, 1/0.2010))

  num_images = images.shape[0]
  fig, axes = plt.subplots(M, N, figsize=(16, 9))
  fig.suptitle('Correct / Predicted Labels (Red text for incorrect ones)')

  for i, ax in enumerate(fig.axes):
    ax.axis('off')
    if i >= num_images:
      continue
    img, label, prediction = images[i], labels[i], preds[i]
    img = inv_norm(img)
    img = img.permute(1, 2, 0) # (C, M, N) -> (M, N, C)
    label, prediction = label.item(), prediction.item()
    if label == prediction:
      ax.set_title(u'\u2713', color='blue', fontsize=22)
    else:
      ax.set_title(
          'X {}/{}'.format(CIFAR10_LABELS[label],
                          CIFAR10_LABELS[prediction]), color='red')
    ax.imshow(img)
  plt.savefig(RESULT_IMG_PATH, transparent=True)

In [30]:
import numpy as np
import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.debug.metrics as met
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.utils.utils as xu
import torchvision
from torchvision import datasets, transforms

class BasicBlock(nn.Module):
  expansion = 1

  def __init__(self, in_planes, planes, stride=1):
    super(BasicBlock, self).__init__()
    self.conv1 = nn.Conv2d(
        in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(planes)
    self.conv2 = nn.Conv2d(
        planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn2 = nn.BatchNorm2d(planes)

    self.shortcut = nn.Sequential()
    if stride != 1 or in_planes != self.expansion * planes:
      self.shortcut = nn.Sequential(
          nn.Conv2d(
              in_planes,
              self.expansion * planes,
              kernel_size=1,
              stride=stride,
              bias=False), nn.BatchNorm2d(self.expansion * planes))

  def forward(self, x):
    out = F.relu(self.bn1(self.conv1(x)))
    out = self.bn2(self.conv2(out))
    out += self.shortcut(x)
    out = F.relu(out)
    return out


class ResNet(nn.Module):

  def __init__(self, block, num_blocks, num_classes=2):
    super(ResNet, self).__init__()
    self.in_planes = 64

    self.conv1 = nn.Conv2d(
        3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(64)
    self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
    self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
    self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
    self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
    self.linear = nn.Linear(512 * block.expansion, num_classes)

  def _make_layer(self, block, planes, num_blocks, stride):
    strides = [stride] + [1] * (num_blocks - 1)
    layers = []
    for stride in strides:
      layers.append(block(self.in_planes, planes, stride))
      self.in_planes = planes * block.expansion
    return nn.Sequential(*layers)

  def forward(self, x):
    out = F.relu(self.bn1(self.conv1(x)))
    out = self.layer1(out)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    out = F.avg_pool2d(out, 4)
    out = torch.flatten(out, 1)
    out = self.linear(out)
    return F.log_softmax(out, dim=1)


def ResNet18():
  return ResNet(BasicBlock, [2, 2, 2, 2])

In [31]:
UNLABELED

'sample@2000'

In [32]:
from src.utils import (
    init_distributed,
    WarmupCosineSchedule,
    CSVLogger,
    AverageMeter,
)

In [ ]:
############## DEBUG #################

In [ ]:
transform, init_transform = make_transforms(
  dataset_name=dataset_name,
  subset_path=subset_path,
  unlabeled_frac=unlabeled_frac,
  training=True,
  split_seed=data_seed,
  crop_scale=crop_scale,
  basic_augmentations=False,
  color_jitter=color_jitter,
  normalize=normalize,
)
multicrop_transform = (multicrop, None)

In [39]:
(
    unsupervised_loader,
    unsupervised_sampler,
    supervised_loader,
    supervised_sampler,
) = init_data(
    dataset_name=dataset_name,
    transform=transform,
    init_transform=init_transform,
    supervised_views=supervised_views,
    u_batch_size=u_batch_size,
    s_batch_size=s_batch_size,
    unique_classes=unique_classes,
    classes_per_batch=classes_per_batch,
    multicrop_transform=multicrop_transform,
    world_size=xm.xrt_world_size(),
    rank=xm.get_ordinal(),
    root_path=root_path,
    s_image_folder=s_image_folder,
    u_image_folder=u_image_folder,
    training=True,
    copy_data=copy_data,
  )

self.multicrop_transform (6, None)
self.multicrop_transform (0, None)


In [40]:
supervised_loader

In [45]:
iter_supervised = iter(supervised_loader)
logger.info(f"len.supervised_loader: {len(iter_supervised)}")
sdata = next(iter_supervised)

In [47]:
ipe = len(unsupervised_loader)

In [48]:
ipe

62

In [52]:
# -- init model
encoder = init_model(
  device=device,
  model_name=model_name,
  use_pred=use_pred_head,
  output_dim=output_dim,
)

# -- init losses
paws = init_paws_loss(multicrop=multicrop, tau=temperature, T=sharpen, me_max=reg)
# -- assume support images are sampled with ClassStratifiedSampler
labels_matrix = make_labels_matrix(
  num_classes=classes_per_batch,
  s_batch_size=s_batch_size,
  world_size=xm.xrt_world_size(),
  device=device,
  unique_classes=unique_classes,
  smoothing=label_smoothing,
) 

In [56]:
scaler = torch.cuda.amp.GradScaler(enabled=use_fp16)
encoder, optimizer, scheduler = init_opt(
      encoder=encoder,
      weight_decay=wd,
      start_lr=start_lr,
      ref_lr=lr,
      final_lr=final_lr,
      ref_mom=mom,
      nesterov=nesterov,
      iterations_per_epoch=ipe,
      warmup=warmup,
      num_epochs=num_epochs,
  )

In [57]:
device = xm.xla_device()

In [59]:
def train_loop_fn(supervised_loader, unsupervised_loader, epoch):
    print("Entre a `train_loop_fn`")

    # -- TRAINING LOOP
    best_loss = None

    logger.info("Epoch %d" % (epoch + 1))

    """
    TO DO: Hacer un Double CHECK EN este Sampler!!!
    """
    # -- update distributed-data-loader epoch
    unsupervised_sampler.set_epoch(epoch)
    if supervised_sampler is not None:
        supervised_sampler.set_epoch(epoch)

    loss_meter = AverageMeter()
    ploss_meter = AverageMeter()
    rloss_meter = AverageMeter()
    time_meter = AverageMeter()
    data_meter = AverageMeter()

    for itr, udata in enumerate(unsupervised_loader):

        def load_imgs():
            # -- unsupervised imgs
            uimgs = [u.to(device, non_blocking=True) for u in udata[:-1]]
            # -- supervised imgs
            global iter_supervised
            try:
                sdata = next(iter_supervised)
            except Exception:
                iter_supervised = iter(supervised_loader)
                logger.info(f"len.supervised_loader: {len(iter_supervised)}")
                sdata = next(iter_supervised)
            finally:
                labels = torch.cat([labels_matrix for _ in range(supervised_views)])
                simgs = [s.to(device, non_blocking=True) for s in sdata[:-1]]
            # -- concatenate supervised imgs and unsupervised imgs
            imgs = simgs + uimgs
            return imgs, labels

        (imgs, labels) = load_imgs()

        def train_step():
            with torch.cuda.amp.autocast(enabled=use_fp16):
                optimizer.zero_grad()

                # --
                # h: representations of 'imgs' before head
                # z: representations of 'imgs' after head
                # -- If use_pred_head=False, then encoder.pred (prediction
                #    head) is None, and _forward_head just returns the
                #    identity, z=h
                h, z = encoder(imgs, return_before_head=True)

                # Compute paws loss in full precision
                with torch.cuda.amp.autocast(enabled=False):

                    # Step 1. convert representations to fp32
                    h, z = h.float(), z.float()

                    # Step 2. determine anchor views/supports and their
                    #         corresponding target views/supports
                    # --
                    num_support = (
                        supervised_views * s_batch_size * classes_per_batch
                    )
                    # --
                    anchor_supports = z[:num_support]
                    anchor_views = z[num_support:]
                    # --
                    target_supports = h[:num_support].detach()
                    target_views = h[num_support:].detach()
                    target_views = torch.cat(
                        [
                            target_views[u_batch_size : 2 * u_batch_size],
                            target_views[:u_batch_size],
                        ],
                        dim=0,
                    )

                    # Step 3. compute paws loss with me-max regularization
                    (ploss, me_max) = paws(
                        anchor_views=anchor_views,
                        anchor_supports=anchor_supports,
                        anchor_support_labels=labels,
                        target_views=target_views,
                        target_supports=target_supports,
                        target_support_labels=labels,
                    )
                    loss = ploss + me_max

            scaler.scale(loss).backward()
            lr_stats = scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            return (float(loss), float(ploss), float(me_max), lr_stats)

        (loss, ploss, rloss, lr_stats) = train_step()
        loss_meter.update(loss)
        ploss_meter.update(ploss)
        rloss_meter.update(rloss)

In [ ]:
######################################

In [42]:
SERIAL_EXEC = xmp.MpSerialExecutor()
# Only instantiate model weights once in memory.
WRAPPED_MODEL = xmp.MpModelWrapper(ResNet18())

def train_resnet18():
  torch.manual_seed(1)

  ############# PAWS CODE ##################

  device = xm.xla_device()

  # -- init model
  encoder = init_model(
    device=device,
    model_name=model_name,
    use_pred=use_pred_head,
    output_dim=output_dim,
  )

  # -- init losses
  paws = init_paws_loss(multicrop=multicrop, tau=temperature, T=sharpen, me_max=reg)
  # -- assume support images are sampled with ClassStratifiedSampler
  labels_matrix = make_labels_matrix(
    num_classes=classes_per_batch,
    s_batch_size=s_batch_size,
    world_size=xm.xrt_world_size(),
    device=device,
    unique_classes=unique_classes,
    smoothing=label_smoothing,
  ) 

  print("normalize {}".format(normalize))

# -- make data transforms
  transform, init_transform = make_transforms(
    dataset_name=dataset_name,
    subset_path=subset_path,
    unlabeled_frac=unlabeled_frac,
    training=True,
    split_seed=data_seed,
    crop_scale=crop_scale,
    basic_augmentations=False,
    color_jitter=color_jitter,
    normalize=normalize,
  )
  multicrop_transform = (multicrop, None)
  if multicrop > 0:
    multicrop_transform = make_multicrop_transform(
        dataset_name=dataset_name,
        num_crops=multicrop,
        size=mc_size,
        crop_scale=mc_scale,
        normalize=normalize,
        color_distortion=color_jitter,
    )

# -- init data-loaders/samplers
  (
    unsupervised_loader,
    unsupervised_sampler,
    supervised_loader,
    supervised_sampler,
) = init_data(
    dataset_name=dataset_name,
    transform=transform,
    init_transform=init_transform,
    supervised_views=supervised_views,
    u_batch_size=u_batch_size,
    s_batch_size=s_batch_size,
    unique_classes=unique_classes,
    classes_per_batch=classes_per_batch,
    multicrop_transform=multicrop_transform,
    world_size=xm.xrt_world_size(),
    rank=xm.get_ordinal(),
    root_path=root_path,
    s_image_folder=s_image_folder,
    u_image_folder=u_image_folder,
    training=True,
    copy_data=copy_data,
  )
  iter_supervised = None
  ipe = len(unsupervised_loader)

  print("len unsuupervised dataloader IS: {}".format(ipe))


  iter_supervised = None
  ipe = len(unsupervised_loader)
  logger.info(f"iterations per epoch: {ipe}")

    # -- init optimizer and scheduler
  scaler = torch.cuda.amp.GradScaler(enabled=use_fp16)
  encoder, optimizer, scheduler = init_opt(
        encoder=encoder,
        weight_decay=wd,
        start_lr=start_lr,
        ref_lr=lr,
        final_lr=final_lr,
        ref_mom=mom,
        nesterov=nesterov,
        iterations_per_epoch=ipe,
        warmup=warmup,
        num_epochs=num_epochs,
    )
  
    #if xm.xrt_world_size() > 1:
    #    encoder = DistributedDataParallel(encoder, broadcast_buffers=False)



  start_epoch = 0

  def train_loop_fn(supervised_loader, unsupervised_loader, epoch):
      print("Entre a `train_loop_fn`")

      # -- TRAINING LOOP
      best_loss = None

      logger.info("Epoch %d" % (epoch + 1))

      """
      TO DO: Hacer un Double CHECK EN este Sampler!!!
      """
      # -- update distributed-data-loader epoch
      unsupervised_sampler.set_epoch(epoch)
      if supervised_sampler is not None:
          supervised_sampler.set_epoch(epoch)

      loss_meter = AverageMeter()
      ploss_meter = AverageMeter()
      rloss_meter = AverageMeter()
      time_meter = AverageMeter()
      data_meter = AverageMeter()

      for itr, udata in enumerate(unsupervised_loader):
          print("Entre a `for itr, udata`")
          def load_imgs():
              # -- unsupervised imgs
              uimgs = [u.to(device, non_blocking=True) for u in udata[:-1]]
              # -- supervised imgs
              global iter_supervised
              try:
                  sdata = next(iter_supervised)
              except Exception:
                  iter_supervised = iter(supervised_loader)
                  logger.info(f"len.supervised_loader: {len(iter_supervised)}")
                  sdata = next(iter_supervised)
              finally:
                  labels = torch.cat([labels_matrix for _ in range(supervised_views)])
                  simgs = [s.to(device, non_blocking=True) for s in sdata[:-1]]
              # -- concatenate supervised imgs and unsupervised imgs
              imgs = simgs + uimgs
              return imgs, labels

          (imgs, labels) = load_imgs()
          
          def train_step():
              print("Entre a train_step()")
              with torch.cuda.amp.autocast(enabled=use_fp16):
                  optimizer.zero_grad()

                  # --
                  # h: representations of 'imgs' before head
                  # z: representations of 'imgs' after head
                  # -- If use_pred_head=False, then encoder.pred (prediction
                  #    head) is None, and _forward_head just returns the
                  #    identity, z=h
                  h, z = encoder(imgs, return_before_head=True)

                  # Compute paws loss in full precision
                  with torch.cuda.amp.autocast(enabled=False):

                      # Step 1. convert representations to fp32
                      h, z = h.float(), z.float()

                      # Step 2. determine anchor views/supports and their
                      #         corresponding target views/supports
                      # --
                      num_support = (
                          supervised_views * s_batch_size * classes_per_batch
                      )
                      # --
                      anchor_supports = z[:num_support]
                      anchor_views = z[num_support:]
                      # --
                      target_supports = h[:num_support].detach()
                      target_views = h[num_support:].detach()
                      target_views = torch.cat(
                          [
                              target_views[u_batch_size : 2 * u_batch_size],
                              target_views[:u_batch_size],
                          ],
                          dim=0,
                      )

                      # Step 3. compute paws loss with me-max regularization
                      (ploss, me_max) = paws(
                          anchor_views=anchor_views,
                          anchor_supports=anchor_supports,
                          anchor_support_labels=labels,
                          target_views=target_views,
                          target_supports=target_supports,
                          target_support_labels=labels,
                      )
                      loss = ploss + me_max

              scaler.scale(loss).backward(). # DOUBLE CHECK HERE, SI ESTA CORDE CON TPU?
              lr_stats = scaler.step(optimizer)
              scaler.update()
              scheduler.step()
              return (float(loss), float(ploss), float(me_max), lr_stats)

          (loss, ploss, rloss, lr_stats) = train_step()
          #(loss, ploss, rloss, lr_stats) = (0,0,0,0)
          loss_meter.update(loss)
          ploss_meter.update(ploss)
          rloss_meter.update(rloss)

  ##########################################

  """
  # Scale learning rate to num cores
  learning_rate = FLAGS['learning_rate'] * xm.xrt_world_size()

  # Get loss function, optimizer, and model
  device = xm.xla_device()
  model = WRAPPED_MODEL.to(device)
  optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                        momentum=FLAGS['momentum'], weight_decay=5e-4)
  loss_fn = nn.NLLLoss()

  def train_loop_fn(loader):
    tracker = xm.RateTracker()
    model.train()
    for x, (data, target) in enumerate(loader):
      optimizer.zero_grad()
      output = model(data)
      loss = loss_fn(output, target)
      loss.backward()
      xm.optimizer_step(optimizer)
      tracker.add(FLAGS['batch_size'])
      if x % FLAGS['log_steps'] == 0:
        print('[xla:{}]({}) Loss={:.5f} Rate={:.2f} GlobalRate={:.2f} Time={}'.format(
            xm.get_ordinal(), x, loss.item(), tracker.rate(),
            tracker.global_rate(), time.asctime()), flush=True)


  # Train and eval loops
  accuracy = 0.0
  data, pred, target = None, None, None
  for epoch in range(1, FLAGS['num_epochs'] + 1):
    para_loader = pl.ParallelLoader(train_loader, [device])
    train_loop_fn(para_loader.per_device_loader(device))
    xm.master_print("Finished training epoch {}".format(epoch))

  return accuracy, data, pred, target
  """
  data, pred, target = None, None, None

  train_supervised_loader = pl.MpDeviceLoader(supervised_loader, device)
  train_unsupervised_loader = pl.MpDeviceLoader(unsupervised_loader, device)

  #for epoch in range(start_epoch, end_epoch):
  for epoch in range(start_epoch, 10):
      print("Epoch is {}".format(epoch))
      #para_loader_1 = pl.ParallelLoader(supervised_loader, [device])
      #para_loader_2 = pl.ParallelLoader(unsupervised_loader, [device])
      # train_loop_fn(para_loader.per_device_loader(device))
      train_loop_fn(train_supervised_loader, train_unsupervised_loader, epoch)
      xm.master_print("Finished training epoch {}".format(epoch))
  return accuracy, data, pred, target

In [ ]:
"""
TO DO: 
  - Ando Teniendo un problema ademas en el train_step() DEBUGGEARLO. 
  - ¿Por que se me esta rompiendo ahi?
"""

In [43]:
# Start training processes
def _mp_fn(rank, flags):
  global FLAGS
  FLAGS = flags
  torch.set_default_tensor_type('torch.FloatTensor')
  accuracy, data, pred, target = train_resnet18()

xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=FLAGS['num_cores'],
          start_method='fork')

normalize True
self.multicrop_transform (6, Compose(
    Compose(
    RandomResizedCrop(size=(96, 96), scale=(0.05, 0.14), ratio=(0.75, 1.3333), interpolation=bilinear)
    RandomHorizontalFlip(p=0.5)
    Compose(
    RandomApply(
    p=0.8
    ColorJitter(brightness=[0.19999999999999996, 1.8], contrast=[0.19999999999999996, 1.8], saturation=[0.19999999999999996, 1.8], hue=[-0.2, 0.2])
)
    RandomGrayscale(p=0.2)
)
    ToTensor()
)
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
))
self.multicrop_transform (0, None)
len unsuupervised dataloader IS: 7
Epoch is 0
Entre a `train_loop_fn`
Entre a `for itr, udata`
Entre a `for itr, udata`
Entre a `for itr, udata`
Entre a `for itr, udata`
Entre a `for itr, udata`
Entre a `for itr, udata`
Entre a `for itr, udata`
Finished training epoch 0
Epoch is 1
Entre a `train_loop_fn`
Entre a `for itr, udata`
Entre a `for itr, udata`
Entre a `for itr, udata`
Entre a `for itr, udata`
Entre a `for itr, udata`
Entre a `for itr, udata`


KeyboardInterrupt: ignored

## Visualize Predictions